# Designing Interpretable Neural Networks
> Generalized Linear and Additive Models are well-established interpretable approaches to supervised learning. This post connects these approaches to the building blocks of Neural Networks, and demonstrates that it's possible to design Neural Networks that are just as interpretable!

- toc: true 
- badges: true
- comments: true
- categories: [transparency]
- image: images/nnflow.png
- show_description: false
- annotations: true
- show_tags: false

In [1]:
#hide
#necessary packages for the notebook
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#custom matplotlib stylesheet for the blog
plt.style.use('blog')

Most supervised prediction problems can be described as learning some function $f$ that minimizes the error-term ($\epsilon$) to the equation $y = f(X) + \epsilon$, where $X$ is the input features and $y$ is the target prediction. This blog post will explore methods to devise neural architectures to learn such functions, with the additional goal of making them as transparent as linear models. However, prior to jumping into such architectures, it's important to understand the foundation of classic transparent approaches, starting from the simplest linear models. 

The next section provides the necessary background to understand how the building blocks of neural networks are related to linear models. Then, we can drop the restriction of linearity with Generalized Additive Models (GAMs), and explore building transparent neural networks.

# Linear Models

Different algorithms are sufficient for learning different families of functions. For example, simple linear models can only learn to make predictions according to functions of the form:

$$y = X^T\beta + \epsilon = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_n X_n + \epsilon$$

Where $\beta_i$ represents learned coefficients with respect to $X_i$.

## Linear Regression

Linear regression is arguably the simplest of the linear models, and comes with four assumptions:

1. **Linearity**: The relationship between $X$ and the mean of $y$ is linear.
2. **Independence**: $X_i$ and $X_j$ are linearly independent of eachother for all $i \neq j$.
3. **Normality**: $y$ given any $X$ comes from a normal distribution.
4. **Homoscedasticity**: The variance of residual is the same for any value of $X$.

These assumptions can be nicely described by one math equation:

$$
\begin{aligned}
y & \in \mathcal{N}(X^T\beta, \sigma^2 I) \\
& \Rightarrow \mathbb{E}[y|X] = X^T\beta
\end{aligned}
$$

However, these assumptions are quite rigid for the real world. Many datasets and/or problem spaces do not conform to these restrictions. So why do we still use linear regression when we have algorithms that can comparably perform the regression task without such rigid assumptions? The common answers to this question are:

1. **Occam's Razor**: Don't add complexity without necessity.    
2. **Little Data**: Ordinary Least Squares (OLS) is a closed form solution to linear regression{% fn 1 %}.
3. **Interpretability**: $y$ can be explained with respect to how $X$ interacts with the $\beta$ coefficients.

Today, we are going to stick with the notion that transparency is of the utmost importance, and assume we have a significant amount of data. Whatever the model is, it must be able to produce feature-wise explanations that are useful. However, contrary to what you may have heard, these models don't need to be exactly linear as described above in order to be comparably interpretable.

## Generalized Linear Models

There are three components to any Generalized Linear Model (GLM):

1. **Random Component**: The probability distribution of $y$ (typically belonging to the exponential family{% fn 2 %}).
2. **Systematic Component**: the right side of the equation for predicting $y$ (often $X^T\beta$).
3. **Link Function**: A function $g$ that links the systematic component and the random component.

This yields the following general equation for GLMs:

$$g(\mathbb{E}[y|X]) = X^T\beta + \epsilon$$

Observe that if the random component is a normal distribution with a constant variance, and the link function is the identity function ($g(y) = y$), then the GLM is exactly linear regression! Hence the functions that GLMs can describe are simply a superset of the functions linear regression can describe. Furthermore, recognize that the random component loosens the constraint of normality, and that the link function alters the constraint of linearity. The relationship between $X$ and the mean of $y$ can be non-linear as long as the relationship between $X$ and the mean of $g(y)$ is linear. Lastly, the residuals are allowed to be heteroscedastic. The only restriction from linear regression that fully remains is the independence of the explanatory variables. 

The link function is the most important aspect to any GLM. The intuition behind a link function is that it transforms the distribution of $y$ to the range $(-\infty,+\infty)$. Link functions are selected according to the given random component.

For example, binary logistic regression assumes the probability distribution of $y$ is a bernoulli distribution. This means $\mu(x)$ is between 0 and 1. We need some function $g: [0,1] \rightarrow \Reals$, and a logit function is sufficient for this:

$$g(\mu) = log(\frac{\mu}{1 - \mu})$$

Given the expected distribution of your problem, you determine a link function that will properly transform your data to the right space. And then you can fit a simple linear model: $g(y) = X^T\beta + \epsilon$. However, Ordinary Least Squares may no longer have a closed form solution in this case. Hence, learning $\beta$ requires a different optimization method: Maximum Likelihood Estimation (MLE).

## Relating to Neural Networks

Neural networks are algorithms built of components called layers. Layers are built of components called nodes. The first layer of a neural network is called the input layer, because each node passes an input feature. The last layer of a neural network is called the output layer, and it should represent the output you are trying to predict (this layer has one node in the classic regression case). Lastly, any layers between the input and output layers are called hidden layers.

This structure of layers is ordered and connected in such a manner that each layer receives information from the previous layer, and passes it to the next. The output $X_j$ of node $j$ in layer $L_i$ gets passed to every node in the next layer in the network, $L_{i + 1}$. The image below displays a neural network with one hidden layer, and highlights the flow through a single node of that hidden layer.

![](https://ryansaxe.com/images/nnflow.png) 

Each node contains some set of weights ($w$) and a bias ($b$). When the node receives the output ($X$) of all the nodes in the preceding layer, it performs the following computation: $X^Tw + b$. Then, before passing this output to the next layer, it gets passed to something called an activation function, which computes some (often non-linear) transform of $X^Tw + b$. The activation function is depicted as $f$ in the image above.

If we substitute our definitions to $w = \big < \beta_1, \beta_2, \cdots, \beta_n \big >$ and $b = \beta_0$, we are left with:

$$
\begin{aligned}
& y = f(X^T\beta) \\
& f^{-1}(y) = X^T\beta
\end{aligned}
$$

Does this look familiar? It's almost exactly the general equation of a GLM. In fact, if $f^\inv = g$, where $g$ is a link function for a GLM, then the computation of a single node on a neural network is identical to a GLM on the output of the previous layer! 

This means that a neural network with zero hidden layers and a linear activation function on the output layer is exactly equivalent to linear regression. And, if we change the activation function to the inverse of the logit function (this is the sigmoid activation function), this neural network becomes exactly equivalent to logistic regression! The code below is a simple prototype of building linear and logistic regression with [Keras](https://www.keras.io), and tests it on a simulated dataset.

In [2]:
#hide
#set up parameters to generate 3 normally distributed columns
columns = list('abc')
beta = [1, 1, 1]
bias = 0.0
size = 100000
mean = 5.0
std = 1.0
#create the dataset
data = {
    col: np.random.normal(
        size=size, loc=mean, scale=std
    ) for col in columns
}
X = pd.DataFrame(data)
betaX = X.copy()
for i,col in enumerate(columns):
    betaX[col] = X[col] * beta[i]
y_logistic = ((bias + betaX.sum(1)) < betaX.sum(1).mean()).astype(float)
y_linear = bias + betaX.sum(1)
#X.assign(target=y).head()

In [3]:
cutoff = betaX.sum(1).mean()
print(cutoff)

14.996218003352118


In [4]:
class Regression(tf.keras.Model):
    def __init__(
        self,
        name=None,
        style='linear'
    ):
        super().__init__(name=name)
        # linear regression uses the identity as the link function
        #    and the inverse of the identity is a linear activation
        if style.lower() == 'linear':
            activation = 'linear'
        # logistic regression uses the logit link function, and the
        #    inverse of logit is a sigmoid activation function
        elif style.lower() == 'logistic':
            activation = 'sigmoid'
        # no other options are supported
        else:
            raise ValueError('input style only supports two options: linear or logistic')
        # pass input directly to the output layer --- no hidden layers
        self.output_layer = tf.keras.layers.Dense(1, activation=activation)
    
    def call(self, x, training=None):
        return self.output_layer(x)

In [5]:
#hide
#logistic regression setup
logistic_reg = Regression(style='logistic')
logistic_reg.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['mean_absolute_error']
)
hist = logistic_reg.fit(
    X.to_numpy(),
    y_logistic.to_numpy(),
    epochs=20,
    batch_size=32,
)

Train on 100000 samples
Epoch 1/20
100000/100000 [==============================] - 4s 36us/sample - loss: 1.1346 - mean_absolute_error: 0.5068
Epoch 2/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.6683 - mean_absolute_error: 0.4848
Epoch 3/20
100000/100000 [==============================] - 3s 34us/sample - loss: 0.6145 - mean_absolute_error: 0.4565
Epoch 4/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.5672 - mean_absolute_error: 0.4293
Epoch 5/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.5263 - mean_absolute_error: 0.4044
Epoch 6/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.4910 - mean_absolute_error: 0.3818
Epoch 7/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.4600 - mean_absolute_error: 0.3612
Epoch 8/20
100000/100000 [==============================] - 3s 33us/sample - loss: 0.4331 - mean_absolute_error: 0.3428
Epoch 9/20
10000

In [6]:
#hide
print('binary crossentropy loss:',round(hist.history['loss'][-1],3),'\nmean absolute error:',round(hist.history['mean_absolute_error'][-1],3))

binary crossentropy loss: 0.269 
mean absolute error: 0.221


In [7]:
#hide
#linear regression setup
linear_reg = Regression(style='linear')
linear_reg.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)
hist = linear_reg.fit(
    X.to_numpy(),
    y_linear.to_numpy(),
    epochs=20,
    batch_size=32,
)

Train on 100000 samples
Epoch 1/20
100000/100000 [==============================] - 4s 35us/sample - loss: 11.1655 - mean_absolute_error: 2.0211
Epoch 2/20
100000/100000 [==============================] - 3s 32us/sample - loss: 0.2649 - mean_absolute_error: 0.3965
Epoch 3/20
100000/100000 [==============================] - 3s 31us/sample - loss: 0.0158 - mean_absolute_error: 0.0888
Epoch 4/20
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0013 - mean_absolute_error: 0.0272
Epoch 5/20
100000/100000 [==============================] - 3s 32us/sample - loss: 2.7817e-05 - mean_absolute_error: 0.0028
Epoch 6/20
100000/100000 [==============================] - 3s 32us/sample - loss: 2.2440e-11 - mean_absolute_error: 2.0603e-06
Epoch 7/20
100000/100000 [==============================] - 3s 31us/sample - loss: 1.8486e-07 - mean_absolute_error: 1.5206e-04
Epoch 8/20
100000/100000 [==============================] - 3s 32us/sample - loss: 1.8091e-07 - mean_absolute_error

In [8]:
#hide
print('mean squared error:',round(hist.history['loss'][-1],3),'\nmean absolute error:',round(hist.history['mean_absolute_error'][-1],3))

mean squared error: 0.0 
mean absolute error: 0.0


In [ ]:
#hide_input
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
fig, axes = plt.subplots(1, 2)
X['target'] = betaX.sum(1)
x_sorted = X.sort_values(by='target')
y_before_label = bias + x_sorted['target']
test_y = (y_before_label < x_sorted['target'].mean()).astype(float).to_numpy()
test_X = x_sorted[columns].to_numpy(dtype=np.float32)

axes[0].plot(y_before_label, linear_reg(test_X), label="Predicted", lw=2)
axes[0].plot(y_before_label,y_before_label,label="Ground Truth", alpha=0.5)
axes[0].legend()
axes[0].set_title('$y = \\sum_{i=1}^3 X_i$', color="black", fontsize=30)
axes[0].set_xlabel('Sum of input features', c='white')
axes[0].set_ylabel('Target feature', c='white')

axes[1].plot(y_before_label, logistic_reg(test_X), label="Predicted", lw=2)
axes[1].plot(y_before_label,test_y,label="Ground Truth", alpha=0.5)
axes[1].plot([y_before_label.min(),y_before_label.max()], [0.5, 0.5], ls="--", lw=1, label="cutoff")
axes[1].legend()
logreg_title = r"$f(x) = \begin{cases} 1, & \sum_{i=1}^3 X_i < " + str(round(cutoff,2)) + r"\\ 0, & otherwise \end{cases}$"
print(logreg_title)
axes[1].set_title(r"$f(x) = \begin{cases} 1, & \sum_{i=1}^3 X_i < "
                  f'{round(cutoff,2)}'
                  r"\\ 0, & otherwise \end{cases}$", fontsize=30)
axes[1].set_xlabel('Sum of input features', c='white')
axes[1].set_ylabel('Target feature', c='white')
plt.show()

$f(x) = \begin{cases} 1, & \sum_{i=1}^3 X_i < 15.0\\ 0, & otherwise \end{cases}$


However, It isn't always as simple as the cases of linear and logistic regression. Not all activation functions are invertible (e.g. ReLU), and hence add non-linearities in ways that are not consistent with GLMs. At the same time, the principal of the link function is to transform $y$ to the proper space, and ReLU does accomplish this under the assumption that $y$ cannot be negative.

There is clearly an intimate connection between neural networks and linear models, as the computational components of neural networks are quite literally non-linear transforms on linear models. So, why are neural networks opaque and GLMs transparent? Let's look at the math for an arbitrarily defined neural network with $k$ hidden layers.

$$
\begin{aligned}
L_0& = \big < X_1, X_2, \cdots, X_n \big > \\
L_1 & = f_1 \Big ( \big < L_0^Tw_{1,1} + b_{1,1}, L_0^Tw_{1,2} + b_{1,2}, \cdots \big > \Big) \\
& \vdots \\
L_k & = f_k \Big( \big < L_{k - 1}^Tw_{k,1} + b_{k,1}, L_{k - 1}^Tw_{k,2} + b_{k,2}, \cdots \big > \Big) \\
y & = L_k^T \beta + \epsilon
\end{aligned}
$$

Where $w_{i,j}$ and $b_{i,j}$ are the weights and bias of the j<sup>th</sup> node in layer $L_i$ with activation function $f_i$.

While the final part of the equation is that of a linear model, observe that every element in $L_k$ is dependent on every input feature in $L_0$. This dependency is what creates the opacity of neural networks, not the non-linearity. This can be seen in the next section on Generalized Additive Models, a transparent approach with highly non-linear transforms to the input features while maintaining some notion of independence. This will shed light on how to design neural networks with control over feature dependence!

In [ ]:
#hide
#set up parameters to generate 3 normally distributed columns
columns = ['a','b','c']
size = 1000000
mean = 0.0
#set to 1.333 so that 3 stds is at around 4
std = 1.0
#create the dataset
data = {
    col: np.random.normal(
        size=size, loc=mean, scale=std
    ) for col in columns
}
df = pd.DataFrame(data)
#apply nonlinear transforms to the data
def piecewise_crazy(x):
    x_ = abs(x)
    if x_ > 2:
        return x ** 3
    elif x_ > 1:
        return x ** 4 * 10
    else:
        return x ** 5 * 100
functions = {
    'a': lambda x: piecewise_crazy(x),
    'b': lambda x: x**4 + x**3 - 10 * x**2 + x,
    'c': lambda x: 10 / ((abs(x) ** 0.5) + 1e-1),
}
for col in columns:
    df[f'{col}_transform'] = df[col].apply(functions[col])
#set up a prediction task where the target is a composition of
#    the nonlinear transforms, but the training data is the normally
#    distributed data prior to being transformed    
X = df[[c for c in df.columns if not c.endswith('_transform')]]
y = df[[c for c in df.columns if c.endswith('_transform')]].sum(1)
#X.assign(target=y).head()

# Generalized Additive Models


Generalized Additive Models (GAMs) take another step towards reducing the restrictions within linear models. There are two modifications that GAMs make to classic GLMs, which truly moves from rigid assumptions to flexible modeling:

1. **Allow non-linearity**: GAMs wrap each component $X_i$ with a function $h_k$, where $h_k$ is some learned function that can be non-linear, but must be smooth (differentiable everywhere). It also can be non-parametric.
    
2. **Allow dependence**: Linear models make the assumption that $X_i$ and $X_j$ are linearly independent forall $i \neq j$. Additive models don't have this property, however we assume that which features interact are known apriori. This means that the systematic component can be an equation that contains non-linear feature interaction like $h_k(X_i,X_j, \cdots)$.
    
Hence, equations for GAMs look like this:

$$g(\mu(X)) = \beta_0 + \beta_1 h_1(X_1) + \beta_2 h_2(X_2, X_3) + \cdots + \beta_m h_m(X_n) + \epsilon$$

Technically, this makes GLMs a special case of GAMs where all functions $h$ are the identity function.

## Fitting Smooth Functions

## Priors and Penalties

In [ ]:
#hide_input
fig, axes = plt.subplots(1, len(X.columns))
for f_idx, column in enumerate(X.columns):
    ax = axes[f_idx]
    start = np.quantile(df[column],0.999)
    end = np.quantile(df[column],0.001)
    test_x = np.expand_dims(np.linspace(start,end,100,dtype=np.float32),axis=1)
    function = functions[column]
    true_y = np.asarray([function(x) for x in test_x])
    ax.plot(test_x, true_y, label="true_function", color=(1.0,75.0/255.0,186.0/255.0))
    ax.legend()

In [ ]:
from pygam import LinearGAM
from pygam import s as spline
#fit to 20 splines of degree 3, with a low smoothing penalty 
# and no further constraints to each of the three features
gam = LinearGAM(spline(0) + spline(1) + spline(2)).fit(X, y)

In [ ]:
#hide
mae = tf.keras.metrics.MeanAbsoluteError()
mse = tf.keras.metrics.MeanSquaredError()
yhat = gam.predict(X)

In [ ]:
#hide_input
print('mean squared error:',round(mse(yhat, y).numpy(),3),'\nmean absolute error:',round(mae(yhat, y).numpy(),3))

In [ ]:
#hide_input
fig, axes = plt.subplots(1, len(X.columns), figsize=(20,5))
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    ax = axes[i] #works because intercept is always last term
    start = np.quantile(df[column],0.001)
    end = np.quantile(df[column],0.999)
    test_x = np.expand_dims(np.linspace(start,end,1000,dtype=np.float32),axis=1)
    function = functions[columns[i]]
    true_y = np.asarray([function(x) for x in test_x])
    XX = gam.generate_X_grid(term=i)
    XX = XX[np.where((XX[:,i] > start) & (XX[:,i] < end))[0],:]
    pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)
    ax.plot(XX[:, term.feature], pdep, label='prediction')
    ax.plot(test_x, true_y, label="true_function", alpha=0.5)
    ax.set_title(repr(term))
    ax.legend()
plt.show()

# Interpretable Neural Networks

## Additive Neural Networks

![](https://ryansaxe.com/images/nam.png)

In [ ]:
from tensorflow.keras import layers, regularizers

#define a simple multi-layer perceptron
class MLP(tf.keras.Model):
    def __init__(
        self,
        name=None
    ):
        super().__init__(name=name)
        # relu helps learn more jagged functions if necessary.
        self.l1 = layers.Dense(8, activation='relu')
        # softplus helps smooth between the jagged areas from above
        #     as softplus is referred to as "SmoothRELU"
        self.l2 = layers.Dense(8, activation='softplus')
        self.l3 = layers.Dense(8, activation='softplus')
        self.l4 = layers.Dense(8, activation='softplus')
        self.output_layer = layers.Dense(1)
    
    def call(self, x, training=None):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        return self.output_layer(x)
    
#define a Neural Additive Model for n features
class NAM(tf.keras.Model):
    def __init__(
        self,
        n_features,
        name=None
    ):
        super().__init__(name=name)
        self.n_features = n_features
        # initialize MLP for each input feature
        self.components = [MLP() for i in range(n_features)]
        # create final layer for a linear combination of learned components
        self.linear_combination = tf.keras.layers.Dense(1)
    
    @tf.function
    def call(self, x, training=None):
        #split up by individual features
        individual_features = tf.split(x, self.n_features, axis=1)
        components = []
        #apply the proper MLP to each individual feature
        for f_idx,individual_feature in enumerate(individual_features):
            component = self.components[f_idx](individual_feature)
            components.append(component)
        #concatenate learned components and return linear combination of them
        components = tf.concat(components, axis=1)
        return self.linear_combination(components)

In [ ]:
#hide_output
model = NAM(n_features=3)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)
hist = model.fit(
    X.to_numpy(),
    y.to_numpy(),
    epochs=30,
    batch_size=32,
)

In [ ]:
#hide_input
print('mean squared error:',round(hist.history['loss'][-1],3),'\nmean absolute error:',round(hist.history['mean_absolute_error'][-1],3))

In [ ]:
#hide_input
fig, axes = plt.subplots(1, len(X.columns), figsize=(20,5))
for f_idx, column in enumerate(X.columns):
    ax = axes[f_idx]
    multiplier = model.linear_combination.weights[0].numpy()[f_idx]
    start = np.quantile(df[column],0.999)
    end = np.quantile(df[column],0.001)
    test_x = np.expand_dims(np.linspace(start,end,100,dtype=np.float32),axis=1)
    function = functions[column]
    true_y = np.asarray([function(x) for x in test_x])
    pred_y = model.components[f_idx](test_x).numpy() * multiplier
    ax.plot(test_x, pred_y, label='predicted_function')
    ax.plot(test_x, true_y, label='true_function', alpha=0.5)
    relu_outs = model.components[f_idx].l1(test_x)
    vlines = 0
    for i in range(8):
        relu_out = relu_outs[:, i]
        nonzeros = np.where(relu_out)[0]
        if len(nonzeros) == 0:
            continue
        start = nonzeros.min()
        end = nonzeros.max()
        if start == 0 and end == 99:
            continue
        elif start == 0:
            val = test_x[end + 1][0]
        else:
            val = test_x[start - 1][0]
        ax.axvline(x=val, alpha=0.6, color=(186.0/255.0,1.0,75.0/255.0), ls="--", lw=1)
        vlines += 1
    title = f'function for column {column}'
    ax.set_title(title)
    relu_breaks = plt.Line2D((0,1),(0,0), color=(186.0/255.0,1.0,75.0/255.0), ls="--", lw=1)
    display = (0,1,2)

    handles, labels = ax.get_legend_handles_labels()

    #Create legend from custom artist/label lists
    ax.legend([handle for i,handle in enumerate(handles) if i in display]+[relu_breaks],
              [label for i,label in enumerate(labels) if i in display]+['RELU Breakpoints'])
    
plt.show()

At first glance, these results look good, but not perfect. While the Neural Additive Model was capable of learning the general shapes of the functions, it looks like it's off on the intercepts for all of them. This should not discount the validity of the model. The corresponding math demonstrates perfectly fitting intercepts of an additive model cannot be guaranteed.

Let $h_i(X_i) = \alpha_i + f(X_i)$ where $f(X_i)$ represents all of the aspects of $h(X_i)$ that dependent on $X_i$, and $\alpha_i$ represents the intercept.

$$
\begin{aligned}
y & = \beta_0 + \sum_{i=1}^n \beta_i h_i(X_i) \\
& = \beta_0 + \sum_{i=1}^n \beta_i (\alpha_i + f_i(X_i)) \\
& = \beta_0 + \sum_{i=1}^n \beta_i \alpha_i + \sum_{i=1}^n \beta_i f_i(X_i)
\end{aligned}
$$

The only way to tease apart these intercepts is via $\beta$. Imagine the proper fit of this equation, for every $i$, had $\beta_i = 1$ and $\alpha_i = 2$. In this case, if half of the learned $\alpha_i$s are zero, and the other half are four, that would yield the exact same result for $\sum_{i=1}^n \beta_i \alpha_i$ as the proper fit. Hence, by way of contradictory example, it is impossible to guarantee learning correct intercepts for any additive model.

However, the "goodness of fit" of this model to the true functions can be shown by comparing the partial derivatives. This is because, when taking a derivative with respect to $X$, any aspects of the equation that don't depend on $X$ (e.g. the intercepts) become zero.

In [ ]:
#hide
#y &= \beta_0 + \sum_{i=1}^n \beta_i \alpha_i + \sum_{i=1}^n \beta_i f_i(X_i)

def dxdy(x, y):
    return (x[1:] - x[:-1])/(y[1:] - y[:-1])

In [ ]:
#hide_input
fig, axes = plt.subplots(1, len(X.columns), figsize=(20,5))
#fig2, axes2 = plt.subplots(1, len(X.columns), figsize=(20,5))
for f_idx, column in enumerate(X.columns):
    ax = axes[f_idx]
    multiplier = model.linear_combination.weights[0].numpy()[f_idx]
    start = np.quantile(df[column],0.999)
    end = np.quantile(df[column],0.001)
    test_x = np.expand_dims(np.linspace(start,end,100,dtype=np.float32),axis=1)
    function = functions[column]
    true_y = np.asarray([function(x) for x in test_x])
    pred_y = model.components[f_idx](test_x).numpy() * multiplier
    relu_outs = model.components[f_idx].l1(test_x)
    vlines = 0
    for i in range(8):
        relu_out = relu_outs[:, i]
        nonzeros = np.where(relu_out)[0]
        if len(nonzeros) == 0:
            continue
        start = nonzeros.min()
        end = nonzeros.max()
        if start == 0 and end == 99:
            continue
        elif start == 0:
            val = test_x[end + 1][0]
        else:
            val = test_x[start - 1][0]
        ax.axvline(x=val, alpha=0.5, color=(186.0/255.0,1.0,75.0/255.0), ls="--", lw=1)
        vlines += 1
    ax.plot(test_x[1:], dxdy(pred_y,test_x), label='predicted_function')
    ax.plot(test_x[1:], dxdy(true_y,test_x), label='true_function',alpha=0.5)
    title = f'derivative for column {column} with {vlines} learned breakpoints'
    ax.set_title(title)
    relu_breaks = plt.Line2D((0,1),(0,0), color=(186.0/255.0,1.0,75.0/255.0), ls="--", lw=1)
    display = (0,1,2)

    handles, labels = ax.get_legend_handles_labels()

    #Create legend from custom artist/label lists
    ax.legend([handle for i,handle in enumerate(handles) if i in display]+[relu_breaks],
              [label for i,label in enumerate(labels) if i in display]+['RELU Breakpoints'])
plt.show()

## Neural Additive Models

## Generalizations

This is the fun part. This is where all your creativity for problem solving given domain knowledge yields an extremely flexible, powerful, and interpretable model!

{{ "The closed form solution for OLS is $\beta = (X^TX)^\inv X^Ty$. This requires $X^TX$ to be invertible, which is the case when the elements in $X$ are linearly independent. This is satisfied by our assumption of independence. Without this assumption, there is no closed form solution, and $\beta$ can be approximated by the maximum likelihood estimation function: $min_\beta(y - \beta X)^T(y - \beta X)$." | fndetail: 1 }}

{{ 'The exponential family is a particular family of probability distributions such that their probability density function (PDF) can be writted as: $P(x | \theta) = f(x) g(\theta) exp \Big( \eta(\theta) \centerdot T(x) \Big)$, where $f$, $g$, $\eta$, and $T$ are known functions and $\theta \in \Reals$ is the only parameter to the PDF.' | fndetail: 2 }}